In [1]:
"""implement transfer learning using cifar10"""
import tensorflow.keras as K
import tensorflow as tf
import numpy as np

In [2]:
def preprocess_data(X, Y):
    """Preprocess Data."""
    Y = tf.one_hot(Y, 10)
    return (K.applications.resnet50.preprocess_input(X),
            tf.reshape(Y, [Y.shape[0], 10]))

In [3]:
def create_model():
    """Create model from resnet50."""
    base_model = K.applications.ResNet50(
            weights='imagenet',
            include_top=True,
            pooling='max')

    old_input = K.Input((32, 32, 3))
    pre = K.layers.Lambda(lambda x: tf.image.resize(x, [224, 224]))(old_input)
    inputs = base_model(pre)
    outputs = K.layers.Dense(10, activation='softmax')(inputs)
    model = K.Model(old_input, outputs)

    return model, base_model

In [4]:
(X_train, Y_train), (X_valid, Y_valid) = K.datasets.cifar10.load_data()

X_train, Y_train = preprocess_data(X_train, Y_train)
X_valid, Y_valid = preprocess_data(X_valid, Y_valid)

model, base_model = create_model()
model.compile(optimizer=K.optimizers.Adam(1e-5),
              loss=K.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()
model.fit(x=X_train, y=Y_train, validation_data=(X_valid, Y_valid),
          batch_size=64, epochs=2)
base_model.trainable = False
model.compile(optimizer=K.optimizers.Adam(),
              loss=K.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()
model.fit(x=X_train, y=Y_train, validation_data=(X_valid, Y_valid),
          batch_size=64, epochs=4)
model.save('cifar10.h5')

102967424/102967424 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 1000)              25636712  
                                                                 
 dense (Dense)               (None, 10)                10010     
                                                                 
Total params: 25646722 (97.83 MB)
Trainable params: 25593602 (97.63 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________
Epoch 1/2
782/782 [==============================] - 561s 664ms/step - loss: 2.2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
  # Evaluate the model
  test_loss, test_acc = model.evaluate(X_valid, Y_valid)
  print(f'Test accuracy: {test_acc}')

  # Calculate F1 Score
  from sklearn.metrics import f1_score, classification_report # Import necessary functions
  Y_pred = model.predict(X_valid)
  Y_pred_classes = np.argmax(Y_pred, axis=1)
  Y_true = np.argmax(Y_valid, axis=1)

  f1 = f1_score(Y_true, Y_pred_classes, average='weighted')
  print(f'F1 Score: {f1}')
  print(classification_report(Y_true, Y_pred_classes))

313/313 [==============================] - 29s 92ms/step - loss: 0.4342 - accuracy: 0.9106
Test accuracy: 0.9106000065803528
313/313 [==============================] - 29s 93ms/step
F1 Score: 0.9109227388665145
              precision    recall  f1-score   support

           0       0.89      0.94      0.91      1000
           1       0.96      0.95      0.96      1000
           2       0.93      0.90      0.91      1000
           3       0.78      0.91      0.84      1000
           4       0.89      0.93      0.91      1000
           5       0.85      0.80      0.82      1000
           6       0.97      0.95      0.96      1000
           7       0.97      0.84      0.90      1000
           8       0.94      0.96      0.95      1000
           9       0.95      0.92      0.94      1000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000

